In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder \
    .appName("DataAnalysis") \
    .master("local[*]") \
    .config("spark.driver.host", "127.0.0.1") \
    .getOrCreate()

data = spark.read.csv('hypothesis_testing_data.csv', header=True, inferSchema=True)

data.show()

data.head()


+------------------+------------------+---+
|            Salary|        ExamScores|Age|
+------------------+------------------+---+
| 59967.14153011233|53.769438869243785| 19|
| 53617.35698828816| 68.69032015851961| 43|
| 61476.88538100693| 69.85928225209845| 34|
| 70230.29856408025| 62.96584096167572| 57|
| 52658.46625276664| 72.58071432500986| 50|
|52658.630430508194| 81.06076285221808| 26|
| 70792.12815507391|103.29278851815795| 60|
| 62674.34729152909| 77.61866719247759| 65|
| 50305.25614065048| 78.86325586084146| 56|
| 60425.60043585965|  73.8833112635075| 46|
| 50365.82307187538| 46.21843177051438| 59|
| 50342.70246429743| 74.60229186826174| 43|
| 57419.62271566034| 75.90345314911539| 52|
|35867.197553422026|111.94863168727929| 42|
| 37750.82167486967| 72.11458552828316| 41|
| 49377.12470759027| 79.52321013500419| 30|
| 44871.68879665576| 74.47932345442135| 24|
| 58142.47332595274| 57.46982943570702| 53|
| 45919.75924478789| 92.14234221772531| 62|
|40876.962986647086| 86.27899549

Row(Salary=59967.14153011233, ExamScores=53.769438869243785, Age=19)

In [ ]:
from pyspark.sql.functions import *
missing_values = data.select([count(when(col(c).isNull(), c)).alias(c) for c in data.columns])
missing_values.show()


In [ ]:
from pyspark.sql.functions import mean

for column in data.columns:
    if dict(data.dtypes)[column] in ['int', 'double']:
        mean_value = data.select(mean(col(column))).collect()[0][0]
        data = data.fillna({column: mean_value})

data = data.dropna()


In [ ]:
# Summary statistics
data.describe().show()
# Show column names
print(data.columns)

# Count of rows and columns
print(f"Rows: {data.count()}, Columns: {len(data.columns)}")

In [ ]:
# Replace 'CategoricalColumn' with an actual column name, if any
grouped_data = data.groupBy("Salary").mean()
grouped_data.show(5)


In [ ]:
print(data.columns)

In [ ]:
# Convert to Pandas for hypothesis testing
pandas_data = data.toPandas()

from statsmodels.multivariate.manova import MANOVA

manova = MANOVA.from_formula('ExamScores + Age ~ Salary', data=pandas_data)
print(manova.mv_test())




In [ ]:
print(data.columns)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


X = pandas_data[['Age', 'ExamScores']]  # Features (independent variables)
y = pandas_data['Salary']  # Target (dependent variable)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Create a Linear Regression model and fit it to the training data
model = LinearRegression()
model.fit(X_train, y_train)

# Step 4: Make predictions on the test set
y_pred = model.predict(X_test)

# Step 5: Evaluate the model
mae = mean_absolute_error(y_test, y_pred)  # Mean Absolute Error
mse = mean_squared_error(y_test, y_pred)  # Mean Squared Error
r2 = r2_score(y_test, y_pred)  # R-squared (goodness of fit)

# Print the evaluation results
print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("R-squared (R2):", r2)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


plt.figure(figsize=(8, 6))
sns.scatterplot(x=X_test['Age'], y=y_test, color='blue', label='Actual')
sns.lineplot(x=X_test['Age'], y=y_pred, color='red', label='Predicted', linewidth=2)

plt.title("Age vs Salary with Regression Line")
plt.xlabel("Age")
plt.ylabel("Salary")
plt.legend()
plt.show()


In [ ]:
print(data.columns)